In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output
from bson.json_util import dumps
import base64

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from pymongo import MongoClient

from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "admin"
dbName = "AAC"
shelter = AnimalShelter(username, password, dbName)

df = pd.DataFrame.from_records(shelter.read({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.A([
        html.Img(
            src='data:image/png;base64,{}'.format(encoded_image.decode()),
            style={
                'height':'25%',
                'width':'25%',
                'display':'block',
                'margin-left': "auto",
                'margin-right': "auto",
                'position':'relative',
                'padding-top':0,
                'padding-right':0
            })
    ], href='https://www.snhu.edu'),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Tennyson'))),
    html.Hr(),
    html.Div(    
        dcc.RadioItems(
        id='filter-choices',
        options = [
            {'label' : 'Water Rescue', 'value' : 'WR'},
            {'label' : 'Mountain Rescue', 'value' : 'MR'},
            {'label' : 'Disaster Rescue', 'value' : 'DR'},
            {'label' : 'Reset', 'value' : 'R'}
        ],
        value = 'R',
        labelStyle={'display': 'inline-block'}
    )),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="multi",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10
    ),
    html.Br(),
     html.Hr(),
    html.Div(className='row',
            style={'display' : 'flex'},
             children=[
                 html.Div(
                 id='graph-id',
                 className='col s12 m6',
                 ),    
                 html.Div(
                 id='map-id',
                 className='col s12 m6',
                 )
             ]
            ),
    html.Div(id="query-out"),
    'Tennyson McCalla CS-340 MongoDB Authentication'
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', 'children'),
    [Input('datatable-id', 'derived_viewport_data')])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    names = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    
    return [
        dcc.Graph(
        figure = px.pie(
        data_frame=dff,
        values = values,
        names = names,
        color_discrete_sequence=px.colors.sequential.RdBu,
        width=800,
        height=500
        ))]

@app.callback(Output('datatable-id', 'data'),
              [Input('filter-choices', 'value')
              ])
def on_click(value):
    # start case
    if value == 'R':
        df = pd.DataFrame.from_records(shelter.read({}))
    elif value == 'WR':
        df = pd.DataFrame(list(shelter.read({"$and":[{"sex_upon_outcome":"Intact Female"},
                                                       {"$or":[
                                                           {"breed":"Labrador Retriever Mix"},
                                                           {"breed":"Chesa Bay Retr Mix"},
                                                           {"breed":{ "$regex":"Newfoundland.*","$options": "i"}}]
                                                       },
                                                       {"$and":[{"age_upon_outcome_in_weeks":{"$gte":26}},
                                                               {"age_upon_outcome_in_weeks":{"$lte":156}}]
                                                       }
                                                       ]}
                                            )))
    elif value == 'MR':
        df = pd.DataFrame(list(shelter.read({"$and":[{"sex_upon_outcome":"Intact Male"},
                                                       {"$or":[
                                                           {"breed":"German Shepherd"},
                                                           {"breed":"Alaskan Malamute"},
                                                           {"breed":"Old English Sheepdog"},
                                                           {"breed":"Rottweiler"},
                                                           {"breed":"Siberian Husky"}]
                                                       },
                                                       {"$and":[{"age_upon_outcome_in_weeks":{"$gte":26}},
                                                               {"age_upon_outcome_in_weeks":{"$lte":156}}]
                                                       }
                                                       ]}
                                           )))
    elif value == 'DR':
        df = pd.DataFrame(list(shelter.read({"$and":[{"sex_upon_outcome":"Intact Male"},
                                                       {"$or":[
                                                           {"breed":"Doberman Pinscher"},
                                                           {"breed":"German Shepherd"},
                                                           {"breed":"Golden Retriever"},
                                                           {"breed":"Bloodhound"},
                                                           {"breed":"Rottweiler"}]
                                                       },
                                                       {"$and":[{"age_upon_outcome_in_weeks":{"$gte":26}},
                                                               {"age_upon_outcome_in_weeks":{"$lte":156}}]
                                                       }
                                                       ]}
                                           )))

    return df.to_dict('records')

@app.callback(
    Output('map-id', "children"),
    [
        Input('datatable-id', "derived_viewport_data")
    ])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[30.75,-97.48],
               zoom=10,
               children=[
                   dl.TileLayer(id="base-layer-id"),
                   # Marker with tool tip and popup
                   dl.Marker(position=[30.75,-97.48],
                             children=[
                                 dl.Tooltip(dff.iloc[0,4]),
                                 dl.Popup([
                                     html.H1("Animal Name"),
                                     html.P(dff.iloc[1,9])
                                 ])
                             ])
               ])
    ]

app